<a href="https://colab.research.google.com/github/ramesitexp/DataScience_Zone/blob/main/llm_sentiment_analysis_without_key.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.0 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 2.16.0
    Uninstalling openai-2.16.0:
      Successfully uninstalled openai-2.16.0


In [2]:
import os
import time
import json
import pandas as pd
import matplotlib.pyplot as plt

from openai import OpenAI
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


In [10]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [11]:
tweets_df = pd.read_csv("tweets.csv")
tweets_df.head()


,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [ ]:
tweets_df.isnull().sum()

,0
textID,0
text,1
selected_text,1
sentiment,0


In [12]:
tweets_df = tweets_df.dropna()

In [14]:
tweets_df.isnull().sum()

,0
textID,0
text,0
selected_text,0
sentiment,0


In [15]:
client = OpenAI(timeout=30)

In [17]:
SENTIMENT_SCHEMA = {
    "name": "sentiment_schema",
    "strict": True,
    "schema": {
        "type": "object",
        "properties": {
            "label": {"type": "string", "enum": ["positive", "neutral", "negative"]}
        },
        "required": ["label"],
        "additionalProperties": False
    }
}

In [18]:
import json

def llm_sentiment(text: str) -> str:
    if pd.isna(text) or not str(text).strip():
        return None

    resp = client.responses.create(
        model="gpt-4.1-mini",
        input=[
            {"role": "system", "content": "Classify sentiment as positive, neutral, or negative."},
            {"role": "user", "content": str(text)}
        ],
        text={
            "format": {
                "type": "json_schema",
                "schema": SENTIMENT_SCHEMA["schema"],
                "name": SENTIMENT_SCHEMA["name"]
            }
        },
        temperature=0
    )
    return json.loads(resp.output_text)["label"]

# Sample the DataFrame first to work on 50 texts
N = min(50, len(tweets_df))   # Changed N to 50 as requested
eval_df = tweets_df.sample(N, random_state=42).copy()
print("Working on rows:", len(eval_df))

preds = []
for t in eval_df["text"].tolist():
    preds.append(llm_sentiment(t))
    time.sleep(0.25)   # adjust: 0.15~0.4 depending on stability

eval_df["llm_sentiment"] = preds

Working on rows: 50


In [19]:
eval_df[["text", "sentiment", "llm_sentiment"]].head(10)

,text,sentiment,llm_sentiment
1589,Enjoy! Family trumps everything,positive,positive
10414,--of them kinda turns me off of it all. And ...,negative,negative
6562,Clive it`s my birthday pat me http://apps.fac...,neutral,neutral
2603,congrats hey,positive,positive
4004,is texting,neutral,neutral
1570,Do you have any idea when the (not so) patien...,neutral,neutral
27232,Tell him where...,neutral,neutral
8286,"Ooooh, I`m jealous I might try and get some ...",negative,negative
580,OHSHNAPSSS. is she pissed at blair as usual ?...,neutral,positive
9189,wee. done with advance audit paper,neutral,positive


In [20]:
accuracy_score(eval_df["sentiment"], eval_df["llm_sentiment"])

0.78